In [2]:
import numpy as np
import pandas as pd

In [5]:
df = pd.read_csv('songdata.csv')

In [6]:
df.head(3)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...


In [7]:
df.shape

(57650, 4)

In [9]:
df= df.sample(n=5000).drop('link',axis=1).reset_index(drop=True)

In [11]:
# Cleaning


In [14]:
df['text']=df['text'].str.lower().replace(r'[^\w\s]','').replace(r'\n','',regex=True)

In [15]:
df['text'][0]

"mommy told me she has everything i want  mommy told me that i should not worry 'bout nothing else  mommy told me that i can't do what i want  mommy told me  but she's only thinking about herself  can  i'm thinking about myself  time to leave and it's just as well  'cause i can't help thinking there's something else  wildlife  so i told he that i must have something more than she offered  and she should not worry about my health  i'm a big boy, not her little boy no more  i'm a big boy and i must learn how to protect myself  hold my own when they give me hell  she can worry about herself  tiny eagles must leave their nest  wild horses withstand the test  old lovers become estranged  little boys who can't help but change  could be nothing but who can tell"

In [22]:
import nltk
nltk.download('punkt')
from nltk.stem.porter import PorterStemmer

ps= PorterStemmer()

def tokenization(txt):
    tokens =  nltk.word_tokenize(txt)
    stemming = [ps.stem(w) for w in tokens]

    return " ".join(stemming)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dhruv\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [23]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [25]:
df['text']

0       mommi told me she ha everyth i want mommi told...
1       if it 's love that you 're run from there is n...
2       the first time ever i saw your face i thought ...
3       there wa a time when i would have follow you t...
4       sometim i wonder what thi old world 's comin '...
                              ...                        
4995    i close my eye and pictur the emerald of the s...
4996    miracl happen , you know miracl happen . mirac...
4997    i think , i understand the reason you wo n't s...
4998    and everi time i get to think everi thought of...
4999    it 's time to face the music tonight i came to...
Name: text, Length: 5000, dtype: object

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

tfid = TfidfVectorizer(stop_words='english')
matrix= tfid.fit_transform(df['text'])

In [28]:
matrix.shape

(5000, 17629)

In [29]:
similarity = cosine_similarity(matrix)

In [30]:
similarity

array([[1.        , 0.03167293, 0.01198035, ..., 0.09973911, 0.06016825,
        0.01152872],
       [0.03167293, 1.        , 0.09025957, ..., 0.07254919, 0.03894609,
        0.08086269],
       [0.01198035, 0.09025957, 1.        , ..., 0.00459926, 0.08878237,
        0.09629541],
       ...,
       [0.09973911, 0.07254919, 0.00459926, ..., 1.        , 0.09336894,
        0.06220653],
       [0.06016825, 0.03894609, 0.08878237, ..., 0.09336894, 1.        ,
        0.06620244],
       [0.01152872, 0.08086269, 0.09629541, ..., 0.06220653, 0.06620244,
        1.        ]])

In [31]:
def recommendation(song):
    idx= df[df['song']==song].index[0]
    distances=sorted(list(enumerate(similarity[idx])),reverse=False,key=lambda x:x[1])

    songs=[]
    for i in distances[1:21]:
        songs.append(df.iloc[i[0]].song)
    return songs   

In [32]:
df['song'][0]

'Wildlife'

In [34]:
recommendation('Another Man')


['The First Time Ever I Saw Your Face',
 'Still Crazy After All These Years',
 'Stronger Than Death',
 'Running Free',
 'Thinking About You',
 'Hopeless',
 'Killer',
 'Look At You',
 'Are You There',
 'God Am',
 'Wild Wine',
 'Walter Whitman Where Are You',
 "I Just Can't Wait",
 'Hush, Little Baby',
 'End Of The Line',
 'Moonlight',
 'Egyptian Song',
 'Where I End And You Begin',
 'Prisoner Of Love',
 'Under A Glass Moon']

In [46]:
import pickle

with open('df.pkl', 'wb') as file:
    pickle.dump(df, file)

with open('similarity.pkl', 'wb') as file:
    pickle.dump(similarity, file)